In [ ]:
%matplotlib inline
import matplotlib
import os
from scipy.signal import savgol_filter
import numpy as np
import random
import matplotlib.pyplot as plt
%matplotlib inline  
from dtw import dtw
import pandas as pd
import scipy
from collections import Counter
from scipy.cluster.hierarchy import dendrogram, linkage
import scipy.cluster.hierarchy as hac
from dtw import dtw
from scipy import asarray as ar,exp
from numpy.linalg import norm
from collections import OrderedDict
import os
from scipy.signal import savgol_filter
import numpy as np
from dtaidistance import dtw
import matplotlib.pyplot as plt
from collections import Counter
from scipy.cluster.hierarchy import dendrogram, linkage
import scipy.cluster.hierarchy as hac
from collections import OrderedDict
import pandas as pd
from numpy.linalg import norm
from dtw import dtw
from scipy.optimize import curve_fit
np.seterr(divide='ignore', invalid='ignore')

## Input

In [ ]:
maturity   = 'senior_4_13'
city       = 'barcelona'
smooth     = 'notsmooth'

## Helpers

In [ ]:
def transform_ts(x, maxTT):

    minx = min(x)  
    x    = [xx - minx for xx in x] 
    maxx = max(x)
    x    = [xx * maxTT / maxx for xx in x]
    
    return x


def getBinnedDistribution(x, y, nbins):

    n, bins = np.histogram(x, bins=nbins)
    sy, _   = np.histogram(x, bins=nbins, weights=y)
    mean    = sy/n
    bins    = (_[1:] + _[:-1])/2
    
    return bins, mean


def getBinnedDistribution2(x, y, nbins):

    x = np.asarray(x)
    y = np.asarray(y)
    
    n, bins   = np.histogram(x, bins=nbins)
    sy, _  = np.histogram(x, bins=nbins, weights=y)
    sy2, _ = np.histogram(x, bins=nbins, weights=y*y)
    
    mean = sy/n
    std  = np.sqrt(sy2/n - mean*mean) 
    bins = (_[1:] + _[:-1])/2

    return bins, mean, std


def fitdata(x, y):
    mean = sum(x * y) / sum(y)
    sigma = np.sqrt(sum(y * (x - mean) ** 2) / sum(y))

    def guassianfunc(xVar, a, b, c, d):
        return a * exp(-(xVar - b) ** 2 / (2 * c ** 2)) + d

    popt, _ = scipy.optimize.curve_fit(guassianfunc, x, y, p0=[max(y), mean,         sigma,0])
    
    result = guassianfunc(x, *popt)
    
    slope, intercept, r_value, p_value, std_err = scipy.stats.linregress(result, y)

    return result, popt, p_value


def split_list(a_list):
    quart = len(a_list)//3
    return a_list[:quart+1], a_list[quart:]



def non_increasing(L): # fall
    
    return all(x>=y for x, y in zip(L, L[1:]))


def non_decreasing(L): # all
    return all(x<=y for x, y in zip(L, L[1:]))



### Shape definitions

In [ ]:
def isrise(x, y):
    
    slope, intercept, r_value, p_value, std_err = scipy.stats.linregress(x, y)

    #if non_decreasing(curve) and not isunderdog(curve):
    if r_value > 0.75 and not isunderdog(y):
        return True
    else:
        return False

    
def isfall(x, y):
    
    y2 = [1 - yy for yy in y]
    
    slope, intercept, r_value, p_value, std_err = scipy.stats.linregress(x, y2)

    #if non_increasing(y) and not isephemeral(y):
    if r_value > 0.75 and not isephemeral(y):
        return True
    else:
        return False   
    
    
def isephemeral(curve):

    varStart = np.std(split_list(curve)[0])
    varEnd   = np.std(split_list(curve)[1])  

    crit = 10
    try:
        crit = (curve[0] - min(curve)) / (curve[-1] - min(curve))  
    except:
        pass    


    varStart = np.std(split_list(curve)[0])
    varEnd   = np.std(split_list(curve)[1])  

    if varStart/varEnd > 5  and crit > 5:
        return True
    else:
        return False


def isunderdog(curve):

    varStart = np.std(split_list(curve)[0])
    varEnd   = np.std(split_list(curve)[1])  

    crit = 10
    try:
        crit = (curve[-1] - min(curve)) / (curve[0] - min(curve)) 
    except:
        pass

    if varEnd/varStart > 5 and crit > 5:
        return True
    else:
        return False    

    

def isriseandfall(x, y):
    
    
    x = np.asarray(x)
    y = np.asarray(y)
    crit = (y[-1] - min(y)) / (y[0] - min(y))
    attempts = 0
    itis = False

    
    while attempts < 5:
        try:
            fitted, popt, p_value = fitdata(x,y) 
            y = list(y)
            if popt[0] > 0 and y.index(max(y)) > 0 and y.index(max(y)) < len(y) and crit < 3:
                return True
            break
        except :
            x = np.asarray([xx+random.random()/10.0 for xx in x])
            attempts += 1

    return itis

    
    
def isfallandrise(x, y):
    
    y = np.asarray(y)  
    x = np.asarray([xx+random.random()/5.0 for xx in x])
    y = np.asarray([xx+random.random()/5.0 for xx in y])
   
    crit = 10
    #try:
    crit = (y[-1] - min(y)) / (y[0] - min(y)) 
    #except:
    #    pass
    attempts = 0
    itis = False
    
    while attempts < 5:
        try:
            fitted, popt, p_value = fitdata(x,y)
            y = list(y)
            if popt[0] < 0 and y.index(min(y)) > 0 and y.index(min(y)) < len(y) and crit < 3:
                itis = True
            break
        except :
            x = np.asarray([xx+random.random()/10.0 for xx in x])
            attempts += 1

    return itis


def get_curve_custers(x, y, ax, indicies, ind, res):

    fall        = isfall(x, y)
    rise        = isrise(x, y)
    ephemeral   = isephemeral(y)
    underdog    = isunderdog(y)
    riseandfall = isriseandfall(x, y)
    fallandrise = isfallandrise(x, y)

    bx = ax[indicies[ind]]

    resd                = {}
    resd['fall']        = fall
    resd['rise']        = rise
    resd['ephemeral']   = ephemeral
    resd['underdog']    = underdog
    resd['riseandfall'] = riseandfall
    resd['fallandrise'] = fallandrise 

    dfres = pd.DataFrame(resd.items())
    dfres = dfres.rename(columns = { 0 : 'cluster'})
    dfres = dfres.rename(columns = { 1 : ind})
    dfres.index = dfres.cluster
    dfres = dfres.drop(columns = ['cluster'])
    dfres = dfres.T

    res = res.append (dfres)
    
    return res, resd, bx

## Do raw clustering

In [ ]:
def get_avg_curves(maturity, city, smooth):

    method     = 'complete'
    datafile   = '../../ProcessedData/' + city + '/timeseries/' + maturity + '/avg_curves_'+smooth+'/10'
    outfolder  = '../../ProcessedData/' + city + '/timeseries/' + maturity  
    infile     = '../../ProcessedData/' + city + '/timeseries/senior_timeseries_' + maturity.split('_', 1)[-1] + '.dat'
    venues_ts  = {}  
    lengths    = []
    folderout  = 'NYCBAR/venue_clusters/' + city

    if not os.path.exists(folderout):   
        os.makedirs(folderout)
    

    for ind, line in enumerate(open(infile)):
        fields = line.strip().split('\t')
        venue  = fields[0]
        ts     = fields[1:]
        venues_ts[venue] = ts# savgol_filter(np.asarray([float(fff) for fff in ts  ] ), 5, 3) 
        lengths.append(len(ts))



    INDEX    = str(max([int(k) for k, v in Counter([fff.split('_')[3] for fff in os.listdir(datafile)]).items() if v == 10]))
    files    = [datafile + '/' + f for f in os.listdir(datafile) if INDEX in f]
    f, ax    = plt.subplots(2, 5, figsize=(16, 5)) 
    indicies = [(i,j) for i in range(2) for j in range(5)]
    ys       = set()

    curve             = []
    curvefiles        = {}
    venuefiles        = {}
    numvenues         = []
    venue_curve_files = {}


    for ind, fn in enumerate(files):
        with open(fn) as myfile:                
            x, y = myfile.read().strip().split('\n')
            x    = [float(xx) for xx in x.split('\t')]
            ys.add(y)
            curvefiles[y] = fn
        

    curves, curvefiles2, labelvennum = zip(*[(yyy, curvefiles[yyy], curvefiles[yyy].split('=')[1].replace('.dat', '')) for yyy in list(ys) ])    
    curves = [[float(yy) for yy in y.split('\t')]  for y in curves ]

      
    for kind, ffnn in enumerate(curvefiles2):
        venuefiles[kind] = '../../ProcessedData/' + city + '/timeseries/' + maturity + '/clusters_venues_'+smooth+'/10/venues_in_'+ ffnn.split('/')[-1].replace('avg_curve_','')

    
    
    clusters_ex = []
    
    res = pd.DataFrame()

    for ind, c in enumerate(curves):
        fitted = []
        
        c = [round(ccc,2) for ccc in c]
        
        bcx, bcy = getBinnedDistribution(x, c, len(c))
        
        bcy = [round(ccc,2) for ccc in bcy]

        
        cruvet = c
 
        res, resd, bx = get_curve_custers(bcx, bcy, ax, indicies, ind, res)

        
        bx.plot(bcx, bcy, color = 'crimson') 
        
        try:
            fitted, popt, p_value = fitdata(bcx, bcy) 
            bx.plot(bcx, fitted)
        except:
            pass
    
        for orient in ['bottom', 'top', 'left', 'right']:
            bx.spines[orient].set_color('lightgrey')
        bx.get_xaxis().set_visible(False)
        bx.get_yaxis().set_visible(False)

        
        clust = [k for k, v in resd.items() if v == True]
        
        
        if len(clust) > 0:
            clust = clust[0]
        
        
        
        
        elif len(clust) == 0 and len(fitted) > 0:
            res, resd, bx = get_curve_custers(range(len(fitted)), fitted, ax, indicies, ind, res)
            clust = '\n'.join([k for k, v in resd.items() if v == True])
            clust = [k for k, v in resd.items() if v == True][0]


        print(labelvennum[ind], len([line.strip() for venline in open(venuefiles[ind])]))
            
        filenameout = folderout + '/' + str(ind) + '_' + clust + '_venues.dat'
        fileout     = open(filenameout, 'w')
        
        for venline in open(venuefiles[ind]):
            fileout.write(venline)
 
        fileout.close()
       
        
        clusters_ex += [k for k, v in resd.items() if v == True]
        bx.set_title(str(ind) + ' -- ' + str(labelvennum[ind]) + ', ' + clust)
        
        for fn in files:
            if str(labelvennum[ind]) in fn:
                venue_curve_files[ind] = fn

                


                
    plt.tight_layout()
    plt.savefig(folderout + '/' + city + '_' + smooth + '_original_clusters.png')
    
    return res, set(clusters_ex)
    
    
    


res, clust = get_avg_curves(maturity, city, smooth)
print (len(clust), set(res.keys()).difference(set(clust)))



## VENUES DISTANCE FROM THE CENTROID

#### Get clusters avg curves

In [ ]:
folderout = 'NYCBAR/venue_clusters/' + city
infile    = '../../ProcessedData/' + city + '/timeseries/senior_timeseries_' + maturity.split('_', 1)[-1] + '.dat'
files     =  [f for f in os.listdir(folderout) if '.png' not in f and 'NO_' not in f]
venues_ts = {}  
lengths   = []
clusters  = [ 'rise', 'fallandrise', 'ephemeral', 'fall', 'riseandfall', 'underdog']


for ind, line in enumerate(open(infile)):
    fields = line.strip().split('\t')
    venue  = fields[0]
    ts     = [float(fff) for fff in fields[1:]]
    venues_ts[venue] = ts#savgol_filter(np.asarray([float(fff) for fff in ts  ] ), 5, 3) 
    lengths.append(len(ts))
    
maxl = max(lengths)
clusters_venues = {}
for fn in files:
    cluster = fn.split('_')[1]
    venues  = [line.strip() for line in open(folderout + '/' + fn)]
    
    if cluster not in clusters_venues:
        clusters_venues[cluster] = venues
    else:
        clusters_venues[cluster] += venues
    
  
clusters_avgcurvedata = {}

for ind, cluster in enumerate(clusters):
    
    ttt = []
    sss = []
    venues    = clusters_venues[cluster]
    
    for v in venues:
        y = venues_ts[v]
        x = transform_ts(range(len(y)), maxl)   
        ttt += list(x)
        sss += list(y)

    clusters_avgcurvedata[cluster] = (ttt,sss)
     

#### Get the binning right

In [ ]:
f, ax    = plt.subplots(2, 3, figsize=(16, 7)) 
indicies = [(i,j) for i in range(2) for j in range(3)]
clusters_avgcurves    = {}

for ind, (cluster, (ttt, sss)) in enumerate(clusters_avgcurvedata.items()):
    
    bx, by = getBinnedDistribution(ttt, sss, 5)
    ax[indicies[ind]].plot(bx, by, linewidth = 3, color = 'steelblue')   
    #ax[indicies[ind]].set_ylim([0.6, 1.6])

    print ind, cluster

    if 'and' in cluster:
        result, popt, p_value = fitdata(bx, by)
        slope, intercept, r_value, p_value, std_err = scipy.stats.linregress(result, by)
        r2 = str(round(r_value**2 ,3))
        ax[indicies[ind]].plot(bx, result, color = 'crimson', label = r2)   
        ax[indicies[ind]].legend(loc = 'best', fontsize = 13)

    elif 'fall' in cluster or 'rise' in cluster:
        slope, intercept, r_value, p_value, std_err = scipy.stats.linregress(bx, [1-bb for bb in by])
        r2 = str(round(r_value**2 ,3))
        ax[indicies[ind]].plot(bx, by, linewidth = 3, color = 'steelblue', label = r2)   
        ax[indicies[ind]].legend(loc = 'best', fontsize = 13)
        
    clusters_avgcurves[cluster] = (bx, by)



#### Get distance sorted

In [ ]:
clusters_venues_dist   = {}
clusters_venues_dist_s = {}
minnum = min([len(venues) for venues in clusters_venues.values()])

for cluster, (bx, by) in clusters_avgcurves.items():
    
    clusters_venues_dist[cluster] = []
    venues = clusters_venues[cluster]

    print cluster
    
    for jind, venue in enumerate(venues):

        t1 = np.asarray(by).reshape(-1,1)
        t2 = np.asarray(venues_ts[venue]).reshape(-1,1)
        

        dist, cost, acc, path = dtw(t1, t2, dist=lambda x, y: norm(x - y, ord=1))                
        clusters_venues_dist[cluster].append((venue, dist))       
    
     
    fout        = open(folderout + '/sorted_dist_venues_in_' + cluster + '.dat', 'w')
    venues_dist = sorted(clusters_venues_dist[cluster], key=lambda tup: tup[1])
    
    for (ven, dist) in venues_dist:
        fout.write(ven + '\t' + str(dist) + '\n')
    fout.close()
    
    venues_dist_d = {}
    for (ven, dist) in venues_dist:
        venues_dist_d[ven] = dist
    
    
    clusters_venues_dist_s[cluster] = venues_dist_d


#### Final plot

In [ ]:
f, ax    = plt.subplots(2, 3, figsize=(16, 7)) 
indicies = [(i,j) for i in range(2) for j in range(3)]



for ind, (cluster, (bx, by)) in enumerate(clusters_avgcurves.items()):
    

    
    ax[indicies[ind]].plot(bx, by, linewidth = 3, color = 'steelblue')   
    #ax[indicies[ind]].set_ylim([0.6, 1.6])


    close_venues = clusters_venues_dist_s[cluster][0:125]
    print len(close_venues)
    for (cv, dist) in close_venues: 
        y = venues_ts[cv]
        x = transform_ts(range(len(y)), 11)   
        ax[indicies[ind]].plot(x, y, linewidth = 0.25, color = 'lightgrey', alpha = 0.6)
    
    
    if 'and' in cluster:
        result, popt, p_value = fitdata(bx, by)
        ax[indicies[ind]].plot(bx, result, color = 'crimson') 
        
    elif 'fall' in cluster or 'rise' in cluster:
        slope, intercept, r_value, p_value, std_err = scipy.stats.linregress(bx, [1-bb for bb in by])
        r2 = '$R^2$ = ' + str(round(r_value**2 ,3))
        ax[indicies[ind]].plot(bx, by, linewidth = 3, color = 'steelblue', label = r2)   
        ax[indicies[ind]].legend(loc = 'best', fontsize = 13)

        
        
        

''' london '''
ax[0,0].set_ylim([0.25, 2.65]) 
ax[0,1].set_ylim([0.6, 1.5])    
ax[0,2].set_ylim([0.5, 2.1])    
ax[1,0].set_ylim([0.3, 2.2])    
ax[1,1].set_ylim([0.5, 1.5])    
ax[1,2].set_ylim([0.4, 1.7]) 


''' barcelona
ax[0,0].set_ylim([0.25, 2.35]) 
ax[0,1].set_ylim([0.6, 1.5])    
ax[0,2].set_ylim([0.65, 1.6])    
ax[1,0].set_ylim([0.3, 2.2])    
ax[1,1].set_ylim([0.7, 1.4])    
ax[1,2].set_ylim([0.5, 1.6])  
'''

''' newyork 
ax[0,0].set_ylim([0.25, 2.35]) 
ax[0,1].set_ylim([0.6, 1.5])    
ax[0,2].set_ylim([0.65, 1.6])    
ax[1,0].set_ylim([0.3, 2.2])    
ax[1,1].set_ylim([0.7, 1.4])    
ax[1,2].set_ylim([0.5, 1.6])  
'''


plt.savefig('NYCBAR/CLUSTERS_NEWYORK.png')

# Plot, aggregate

In [ ]:
f, ax     = plt.subplots(2, 3, figsize=(16, 7)) 
folderout = 'NYCBAR/venue_clusters/' + city
infile    = '../../ProcessedData/' + city + '/timeseries/senior_timeseries_' + maturity.split('_', 1)[-1] + '.dat'
files     =  [f for f in os.listdir(folderout) if '.png' not in f and 'NO_' not in f]
venues_ts = {}  
lengths   = []
clusters  = [ 'rise', 'fallandrise', 'ephemeral', 'fall', 'riseandfall', 'underdog']

for ind, line in enumerate(open(infile)):
    fields = line.strip().split('\t')
    venue  = fields[0]
    ts     = fields[1:]
    venues_ts[venue] = ts# savgol_filter(np.asarray([float(fff) for fff in ts  ] ), 5, 3) 
    lengths.append(len(ts))


clusters_venues = {}

for fn in files:
    cluster = fn.split('_')[1]
    venues  = [line.strip() for line in open(folderout + '/' + fn)]
    
    if cluster not in clusters_venues:
        clusters_venues[cluster] = venues
    else:
        clusters_venues[cluster] += venues

        
N_closest = 250


venues    = clusters_venues[cluster]
   


for ind, cluster in enumerate(clusters):
    ijk    = indicies[ind]
    #venues = clusters_venues[cluster]
    
    venues    = clusters_venues_dist_s[cluster]
    venues_s  = sorted([(k, v) for k, v in venues.items()], key = lambda tup: tup[1])[0:N_closest]
    venues_a  = sorted([(k, v) for k, v in venues.items()], key = lambda tup: tup[1])
    venues_s  = [v for (v, d) in venues_s]
    venues_a  = [v for (v, d) in venues_a]
    
    
    fout = open(folderout + '/' + str(ind) + '_FINAL_' + cluster + '.dat', 'w')
    
    
    ttt = []
    sss = []
    
    for venue in venues_s:       
        y = [float(a) for a in venues_ts[venue]]
        x = transform_ts(range(len(y)), 11)    
        ax[indicies[ind]].plot(x, y, linewidth = 0.25, color = 'lightgrey', alpha = 0.6)
        fout.write(venue + '\n')
    fout.close()
    
    
    for venue in venues_a:       
        y = [float(a) for a in venues_ts[venue]]
        x = transform_ts(range(len(y)), 11)  
        ttt += list(x)
        sss += list(y)
    
    
    
    bx, by = getBinnedDistribution(ttt, sss, 7)
    ax[indicies[ind]].plot(bx, by, linewidth = 3, color = 'steelblue') 
    
    
    if 'and' in cluster:
        result, popt, p_value = fitdata(bx, by)
        ax[indicies[ind]].plot(bx, result, color = 'crimson') 
        
    elif 'fall' in cluster or 'rise' in cluster:
        slope, intercept, r_value, p_value, std_err = scipy.stats.linregress(bx, [1-bb for bb in by])
        r2 = '$R^2$ = ' + str(round(r_value**2 ,3))
        ax[indicies[ind]].plot(bx, by, linewidth = 3, color = 'steelblue', label = r2)   
        ax[indicies[ind]].legend(loc = 'best', fontsize = 13)
    
    
    plt.savefig(folderout + '/all_clusters_' + city + '.png')
    
    